## LSTM_use_model2

model1
- senticle데이터만을 사용
- 단어 길이 25

결과
- 부정을 찾지 못 함.
- 긍정은 몇 개 나옴

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -v python-mecab-ko

Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local
sysconfig: /usr
Additional context:
user = False
home

In [ ]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 모델 불러오기
from tensorflow.python.keras.models import load_model
model = load_model('/content/drive/MyDrive/경희대/KHUDA/데비_프로젝트/LSTM/lstm_model_2.h5')
model.summary()

## 뉴스 기사 제목 불러오기
news_df = pd.read_csv('/content/drive/MyDrive/경희대/KHUDA/데비_프로젝트/BERT/samsung_news.csv')
news_df=news_df[['title']]

import mecab
mecab = mecab.MeCab()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 25)          168700    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                23040     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 191,935
Trainable params: 191,935
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def sent_predict (news_df):
  news_df['title'] = [re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',' ',s) for s in news_df['title']] # 한국어만 남기고 싶으면
  news_df['token'] = news_df['title'].apply(mecab.morphs)
  # 불용어 제거
  news_df['remove_stopwords'] = None
  stopwords = ['유로','라고','밝혔','대한','명','된다','부터','에게','등','리','관한','드','한다고','몇','코네','던','노','비','았','입니다','할','적','습니다','는다','니','저','로','것','핀란드','만','어','및','다고','고','된','될','약','게','며','지','합니다','아','또','때','겠','뿐','고','있','수','하','기','년','인','되','면','한다','에서','그','했','다','을','의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
  for i in range(len(news_df)):
    stopwords_removed_sentence = [word for word in news_df.iloc[i,1] if not word in stopwords] # 불용어 제거
    news_df['remove_stopwords'][i] = stopwords_removed_sentence
    input_news = news_df['remove_stopwords']

  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(input_news)
  # 정수 인코딩
  input_news_encoded = tokenizer.texts_to_sequences(input_news)

  # 단어 길이 25으로 padding
  max_len = 25
  pad_new = pad_sequences(input_news_encoded, maxlen = max_len)
  score = model.predict(pad_new)

  news_df['predict'] = None
  news_df['prob'] = None

  for i in range(len(news_df)):
    if score[i].argmax() == 0:
      sent_predict = 0
      prob = ("{:.2f}% 확률로 중성 리뷰입니다.\n".format(score[i][0]/sum(score[i]) * 100))
      news_df['predict'][i] = sent_predict
      news_df['prob'][i] = prob
    elif score[i].argmax() == 1:
      sent_predict = 1
      prob = ("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score[i][1]/sum(score[i]) * 100))
      news_df['predict'][i] = sent_predict
      news_df['prob'][i] = prob
    elif score[i].argmax() == 2:
      sent_predict = 2
      prob = ("{:.2f}% 확률로 부정 리뷰입니다.\n".format(score[i][2]/sum(score[i]) * 100))
      news_df['predict'][i] = sent_predict
      news_df['prob'][i] = prob

  return news_df

In [ ]:
sent_predict(news_df)

,title,token,remove_stopwords,predict,prob
0,삼성 준법위 사업지원 회동 콘트롤타워 부활 논의했나,"[삼성, 준, 법위, 사업, 지원, 회동, 콘트롤, 타워, 부활, 논의, 했, 나]","[삼성, 준, 법위, 사업, 지원, 회동, 콘트롤, 타워, 부활, 논의, 나]",0,54.51% 확률로 중성 리뷰입니다.\n
1,삼성전자 장애인 직원 채용 자회사 만든다 일자리 창출,"[삼성전자, 장애, 인, 직원, 채용, 자회사, 만든다, 일자리, 창출]","[삼성전자, 장애, 직원, 채용, 자회사, 만든다, 일자리, 창출]",0,41.33% 확률로 중성 리뷰입니다.\n
2,삼성전자 자회사형 장애인 표준사업장 만든다,"[삼성전자, 자회사, 형, 장애, 인, 표준, 사업장, 만든다]","[삼성전자, 자회사, 형, 장애, 표준, 사업장, 만든다]",1,41.95% 확률로 긍정 리뷰입니다.\n
3,한국남동발전 삼성전자 탄소중립 함께 달린다,"[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]","[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]",0,73.34% 확률로 중성 리뷰입니다.\n
4,삼성 임원들 격려금 억 모아 헌혈버스 대 기부,"[삼성, 임원, 들, 격려금, 억, 모아, 헌혈, 버스, 대, 기부]","[삼성, 임원, 격려금, 억, 모아, 헌혈, 버스, 대, 기부]",0,78.31% 확률로 중성 리뷰입니다.\n
5,삼성전자 찾은 민주당 기업 미 중 패권 희생양 안되도록 할 것,"[삼성전자, 찾, 은, 민주당, 기업, 미, 중, 패권, 희생양, 안, 되, 도록,...","[삼성전자, 찾, 민주당, 기업, 미, 중, 패권, 희생양, 안, 도록]",0,76.71% 확률로 중성 리뷰입니다.\n
6,세계 등 한국제품 몇개일까 삼성전자가 올킬,"[세계, 등, 한국, 제품, 몇, 개, 일까, 삼성전자, 가, 올, 킬]","[세계, 한국, 제품, 개, 일까, 삼성전자, 올, 킬]",0,83.30% 확률로 중성 리뷰입니다.\n
7,국내 대기업 내 여성 임원 명 돌파 삼성전자 최다,"[국내, 대, 기업, 내, 여성, 임원, 명, 돌파, 삼성전자, 최다]","[국내, 대, 기업, 내, 여성, 임원, 돌파, 삼성전자, 최다]",0,54.29% 확률로 중성 리뷰입니다.\n
8,삼성 대한적십자사에 헌혈버스 대 기증 이재용 동행 철학 실천,"[삼성, 대한, 적십자사, 에, 헌혈, 버스, 대, 기증, 이재용, 동행, 철학, 실천]","[삼성, 적십자사, 헌혈, 버스, 대, 기증, 이재용, 동행, 철학, 실천]",0,75.75% 확률로 중성 리뷰입니다.\n
9,세계 위면 뭐하나 창고에 쌓이는 삼성,"[세계, 위, 면, 뭐, 하나, 창고, 에, 쌓이, 는, 삼성]","[세계, 위, 뭐, 하나, 창고, 쌓이, 삼성]",1,64.05% 확률로 긍정 리뷰입니다.\n


In [ ]:
news_df[news_df['predict'] == 0]

,title,token,remove_stopwords,predict,prob
0,삼성 준법위 사업지원 회동 콘트롤타워 부활 논의했나,"[삼성, 준, 법위, 사업, 지원, 회동, 콘트롤, 타워, 부활, 논의, 했, 나]","[삼성, 준, 법위, 사업, 지원, 회동, 콘트롤, 타워, 부활, 논의, 나]",0,54.51% 확률로 중성 리뷰입니다.\n
1,삼성전자 장애인 직원 채용 자회사 만든다 일자리 창출,"[삼성전자, 장애, 인, 직원, 채용, 자회사, 만든다, 일자리, 창출]","[삼성전자, 장애, 직원, 채용, 자회사, 만든다, 일자리, 창출]",0,41.33% 확률로 중성 리뷰입니다.\n
3,한국남동발전 삼성전자 탄소중립 함께 달린다,"[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]","[한국남, 동발, 전, 삼성전자, 탄소, 중립, 함께, 달린다]",0,73.34% 확률로 중성 리뷰입니다.\n
4,삼성 임원들 격려금 억 모아 헌혈버스 대 기부,"[삼성, 임원, 들, 격려금, 억, 모아, 헌혈, 버스, 대, 기부]","[삼성, 임원, 격려금, 억, 모아, 헌혈, 버스, 대, 기부]",0,78.31% 확률로 중성 리뷰입니다.\n
5,삼성전자 찾은 민주당 기업 미 중 패권 희생양 안되도록 할 것,"[삼성전자, 찾, 은, 민주당, 기업, 미, 중, 패권, 희생양, 안, 되, 도록,...","[삼성전자, 찾, 민주당, 기업, 미, 중, 패권, 희생양, 안, 도록]",0,76.71% 확률로 중성 리뷰입니다.\n
6,세계 등 한국제품 몇개일까 삼성전자가 올킬,"[세계, 등, 한국, 제품, 몇, 개, 일까, 삼성전자, 가, 올, 킬]","[세계, 한국, 제품, 개, 일까, 삼성전자, 올, 킬]",0,83.30% 확률로 중성 리뷰입니다.\n
7,국내 대기업 내 여성 임원 명 돌파 삼성전자 최다,"[국내, 대, 기업, 내, 여성, 임원, 명, 돌파, 삼성전자, 최다]","[국내, 대, 기업, 내, 여성, 임원, 돌파, 삼성전자, 최다]",0,54.29% 확률로 중성 리뷰입니다.\n
8,삼성 대한적십자사에 헌혈버스 대 기증 이재용 동행 철학 실천,"[삼성, 대한, 적십자사, 에, 헌혈, 버스, 대, 기증, 이재용, 동행, 철학, 실천]","[삼성, 적십자사, 헌혈, 버스, 대, 기증, 이재용, 동행, 철학, 실천]",0,75.75% 확률로 중성 리뷰입니다.\n
10,삼성 저격 삼성생명법 년 만의 부활,"[삼성, 저격, 삼성생명, 법, 년, 만, 의, 부활]","[삼성, 저격, 삼성생명, 법, 부활]",0,54.71% 확률로 중성 리뷰입니다.\n
13,품질경영대회 은탑훈장에 년 위 삼성전자 김형남 부사장,"[품질, 경영, 대회, 은, 탑, 훈장, 에, 년, 위, 삼성전자, 김형남, 부사장]","[품질, 경영, 대회, 탑, 훈장, 위, 삼성전자, 김형남, 부사장]",0,74.62% 확률로 중성 리뷰입니다.\n


In [ ]:
news_df[news_df['predict'] == 1]

,title,token,remove_stopwords,predict,prob
2,삼성전자 자회사형 장애인 표준사업장 만든다,"[삼성전자, 자회사, 형, 장애, 인, 표준, 사업장, 만든다]","[삼성전자, 자회사, 형, 장애, 표준, 사업장, 만든다]",1,41.95% 확률로 긍정 리뷰입니다.\n
9,세계 위면 뭐하나 창고에 쌓이는 삼성,"[세계, 위, 면, 뭐, 하나, 창고, 에, 쌓이, 는, 삼성]","[세계, 위, 뭐, 하나, 창고, 쌓이, 삼성]",1,64.05% 확률로 긍정 리뷰입니다.\n
11,세계에서 팔린 대 중 대는 삼성 누적 점유율 위,"[세계, 에서, 팔린, 대, 중, 대, 는, 삼성, 누적, 점유, 율, 위]","[세계, 팔린, 대, 중, 대, 삼성, 누적, 점유, 율, 위]",1,71.61% 확률로 긍정 리뷰입니다.\n
12,삼성전자 식품관 만 방문 기념 고객 감사 기획전,"[삼성전자, 식품, 관, 만, 방문, 기념, 고객, 감사, 기획, 전]","[삼성전자, 식품, 관, 방문, 기념, 고객, 감사, 기획, 전]",1,50.42% 확률로 긍정 리뷰입니다.\n
14,삼성 글로벌 점유율 위 는 년 연속 왕좌 예약,"[삼성, 글로벌, 점유, 율, 위, 는, 년, 연속, 왕좌, 예약]","[삼성, 글로벌, 점유, 율, 위, 연속, 왕좌, 예약]",1,44.57% 확률로 긍정 리뷰입니다.\n
15,삼성전자 메모리 한파에 세계 반도체 위 자리 인텔에 내줬다,"[삼성전자, 메모리, 한파, 에, 세계, 반도체, 위, 자리, 인텔, 에, 내줬, 다]","[삼성전자, 메모리, 한파, 세계, 반도체, 위, 자리, 인텔, 내줬]",1,71.00% 확률로 긍정 리뷰입니다.\n
20,애플 삼성전자 낸드플래시 도입 가능성 견제 영향,"[애플, 삼성전자, 낸드, 플래시, 도입, 가능, 성, 견제, 영향]","[애플, 삼성전자, 낸드, 플래시, 도입, 가능, 성, 견제, 영향]",1,44.86% 확률로 긍정 리뷰입니다.\n
26,종합 강세에도 전 세계 휩쓴 삼성 점유율 절반 차지,"[종합, 강세, 에, 도, 전, 세계, 휩쓴, 삼성, 점유, 율, 절반, 차지]","[종합, 강세, 전, 세계, 휩쓴, 삼성, 점유, 율, 절반, 차지]",1,79.18% 확률로 긍정 리뷰입니다.\n
29,반도체 치킨게임 삼성전자 하이닉스 왜 달라지나,"[반도체, 치킨, 게임, 삼성전자, 하이닉스, 왜, 달라지, 나]","[반도체, 치킨, 게임, 삼성전자, 하이닉스, 왜, 달라지, 나]",1,55.53% 확률로 긍정 리뷰입니다.\n
31,삼성전자 세계 시장 점유율 로 위 수성 프리미엄 부문서 선전,"[삼성전자, 세계, 시장, 점유, 율, 로, 위, 수성, 프리미엄, 부문, 서, 선전]","[삼성전자, 세계, 시장, 점유, 율, 위, 수성, 프리미엄, 부문, 서, 선전]",1,74.15% 확률로 긍정 리뷰입니다.\n


In [ ]:
news_df[news_df['predict'] == 2]

,title,token,remove_stopwords,predict,prob
